In [1]:
import xarray as xr
import os 
import pandas as pd

In [2]:
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
xarrs = [xr.open_dataset(f"./../../data/raw/CDS/cds_{year}.nc") for year in years] 
xarrs[0]


<xarray.Dataset> Size: 513MB
Dimensions:    (longitude: 37, latitude: 33, time: 8760)
Coordinates:
  * longitude  (longitude) float32 148B 6.0 6.25 6.5 6.75 ... 14.5 14.75 15.0
  * latitude   (latitude) float32 132B 55.0 54.75 54.5 54.25 ... 47.5 47.25 47.0
  * time       (time) datetime64[ns] 70kB 2015-01-01 ... 2015-12-31T23:00:00
Data variables:
    u100       (time, latitude, longitude) float64 86MB ...
    v100       (time, latitude, longitude) float64 86MB ...
    v10        (time, latitude, longitude) float64 86MB ...
    d2m        (time, latitude, longitude) float64 86MB ...
    t2m        (time, latitude, longitude) float64 86MB ...
    ssrd       (time, latitude, longitude) float64 86MB ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-06 12:46:06 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [3]:
dfs_years = []
for xarr in xarrs:
    
    xarr = xarr.coarsen(
        latitude=8, 
        longitude=12,
        boundary='trim'  # 'trim' drops any data not fitting into the window perfectly; use 'pad' to include all data.
    ).mean()
    df = xarr.to_dataframe()
    df.reset_index(inplace=True)
    df = df.drop(columns=["v10"]) if "v10" in df.columns else df
    dfs_years.append(df)

df = pd.concat(dfs_years)

<xarray.Dataset> Size: 513MB
Dimensions:    (longitude: 37, latitude: 33, time: 8760)
Coordinates:
  * longitude  (longitude) float32 148B 6.0 6.25 6.5 6.75 ... 14.5 14.75 15.0
  * latitude   (latitude) float32 132B 55.0 54.75 54.5 54.25 ... 47.5 47.25 47.0
  * time       (time) datetime64[ns] 70kB 2015-01-01 ... 2015-12-31T23:00:00
Data variables:
    u100       (time, latitude, longitude) float64 86MB ...
    v100       (time, latitude, longitude) float64 86MB ...
    v10        (time, latitude, longitude) float64 86MB ...
    d2m        (time, latitude, longitude) float64 86MB ...
    t2m        (time, latitude, longitude) float64 86MB ...
    ssrd       (time, latitude, longitude) float64 86MB ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-06 12:46:06 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [4]:
xarrs[1]

<xarray.Dataset> Size: 515MB
Dimensions:    (longitude: 37, latitude: 33, time: 8784)
Coordinates:
  * longitude  (longitude) float32 148B 6.0 6.25 6.5 6.75 ... 14.5 14.75 15.0
  * latitude   (latitude) float32 132B 55.0 54.75 54.5 54.25 ... 47.5 47.25 47.0
  * time       (time) datetime64[ns] 70kB 2016-01-01 ... 2016-12-31T23:00:00
Data variables:
    u100       (time, latitude, longitude) float64 86MB ...
    v100       (time, latitude, longitude) float64 86MB ...
    v10        (time, latitude, longitude) float64 86MB ...
    d2m        (time, latitude, longitude) float64 86MB ...
    t2m        (time, latitude, longitude) float64 86MB ...
    ssrd       (time, latitude, longitude) float64 86MB ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-06 12:07:25 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [6]:
df

,time,latitude,longitude,u100,v100,d2m,t2m,ssrd
0,2015-01-01 00:00:00,54.125,7.375,10.874151,5.961689,277.125047,279.160911,0.0
1,2015-01-01 00:00:00,54.125,10.375,9.032724,2.532830,275.988692,277.586526,0.0
2,2015-01-01 00:00:00,54.125,13.375,10.432904,-0.859244,276.398552,277.750406,0.0
3,2015-01-01 00:00:00,52.125,7.375,6.132433,3.605310,274.006642,274.732349,0.0
4,2015-01-01 00:00:00,52.125,10.375,6.827257,0.254472,275.319620,276.275693,0.0
...,...,...,...,...,...,...,...,...
105115,2023-12-31 23:00:00,50.125,10.375,4.274588,7.341687,275.192887,278.369278,0.0
105116,2023-12-31 23:00:00,50.125,13.375,4.823327,4.181998,274.918440,277.200108,0.0
105117,2023-12-31 23:00:00,48.125,7.375,5.941003,7.190909,275.765710,278.350112,0.0
105118,2023-12-31 23:00:00,48.125,10.375,4.034206,4.521925,273.754287,275.667675,0.0


In [7]:

lats = df["latitude"].unique()
lons = df["longitude"].unique()
dfs = []

for lat in lats:
    for lon in lons:
        dfs.append(df[(df["latitude"] == lat) & (df["longitude"] == lon)].copy())
        dfs[-1].drop(columns=["latitude", "longitude"], inplace=True)
        for column in dfs[-1].columns:
            if column != "time":
                dfs[-1].rename(columns={column: "lat" + str(lat) + "_lon" + str(lon) + "_" + column}, inplace=True)
        dfs[-1].set_index("time", inplace=True)

print(len(dfs))

for i in range(len(dfs)-1):
    dfs[0] = dfs[0].join(dfs[i+1], how="inner")


df_final = dfs[0]
assert df.shape[0] / 12 == df_final.shape[0]
assert (df.shape[1] - 3) * 12 == dfs[0].shape[1]
# 2016 and 2020 are leap years
assert df_final.shape[0] == 24*365* 7 + 24 * 366 * 2
    

12


In [8]:
df_final.head(26)

,lat54.125_lon7.375_u100,lat54.125_lon7.375_v100,lat54.125_lon7.375_d2m,lat54.125_lon7.375_t2m,lat54.125_lon7.375_ssrd,lat54.125_lon10.375_u100,lat54.125_lon10.375_v100,lat54.125_lon10.375_d2m,lat54.125_lon10.375_t2m,lat54.125_lon10.375_ssrd,...,lat48.125_lon10.375_u100,lat48.125_lon10.375_v100,lat48.125_lon10.375_d2m,lat48.125_lon10.375_t2m,lat48.125_lon10.375_ssrd,lat48.125_lon13.375_u100,lat48.125_lon13.375_v100,lat48.125_lon13.375_d2m,lat48.125_lon13.375_t2m,lat48.125_lon13.375_ssrd
time,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,10.874151,5.961689,277.125047,279.160911,0.000000,9.032724,2.532830,275.988692,277.586526,0.000000,...,0.838743,0.501438,268.372486,269.051977,0.000000,2.967056,0.737287,269.147874,269.856141,0.000000
2015-01-01 01:00:00,10.767777,6.487743,277.234200,279.166540,0.000000,9.170908,2.957436,275.938532,277.543080,0.000000,...,0.485406,0.073136,268.090265,268.793064,0.000000,3.079587,0.526616,269.112304,269.872266,0.000000
2015-01-01 02:00:00,10.523435,7.199281,277.257884,279.096960,0.000000,9.104914,3.470364,275.907953,277.454121,0.000000,...,0.038777,-0.111434,267.655796,268.370354,0.000000,3.168738,0.409217,268.986227,269.809585,0.000000
2015-01-01 03:00:00,11.200997,7.373521,277.284468,279.040787,0.000000,8.865877,4.244569,275.909838,277.426427,0.000000,...,-0.365820,-0.192131,267.541363,268.263019,0.000000,3.113468,0.310996,268.975480,269.824801,0.000000
2015-01-01 04:00:00,11.107078,7.778193,277.282412,279.007742,0.000000,9.515645,4.291647,275.887771,277.345608,0.000000,...,-0.503446,-0.166824,267.529296,268.289349,0.000000,2.910088,0.203873,268.828619,269.751037,0.000000
2015-01-01 05:00:00,9.783081,9.599065,277.230214,279.025406,0.000000,9.563067,4.400334,275.876503,277.330485,0.000000,...,-0.596397,-0.162086,267.188772,267.938202,0.000000,2.652896,0.016212,268.897920,269.821444,0.000000
2015-01-01 06:00:00,10.426088,9.257642,277.153103,278.978582,0.000000,8.661054,5.449615,275.814340,277.233076,0.000000,...,-0.890773,0.446803,267.146046,267.930435,0.000000,2.507476,0.053278,268.985670,269.956597,0.000000
2015-01-01 07:00:00,10.482978,9.255194,277.100034,278.935042,0.000000,9.214530,5.679094,275.605675,277.020234,0.000000,...,-1.096775,1.284462,266.948559,267.848076,0.000000,2.435061,0.272669,268.986666,269.946959,44.108424
2015-01-01 08:00:00,10.983852,9.390034,277.060406,278.823317,500.267692,9.191319,5.991008,275.440400,276.829910,3826.824511,...,-0.697087,1.142609,267.261242,268.198066,59475.463578,2.300094,0.258999,269.338736,270.362697,45297.117849


In [11]:

df_final.to_csv("./../../data/raw/CDS/weather.csv")


(78888, 60)
78888


In [12]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 78888 entries, 2015-01-01 00:00:00 to 2023-12-31 23:00:00
Data columns (total 60 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lat54.125_lon7.375_u100   78888 non-null  float64
 1   lat54.125_lon7.375_v100   78888 non-null  float64
 2   lat54.125_lon7.375_d2m    78888 non-null  float64
 3   lat54.125_lon7.375_t2m    78888 non-null  float64
 4   lat54.125_lon7.375_ssrd   78888 non-null  float64
 5   lat54.125_lon10.375_u100  78888 non-null  float64
 6   lat54.125_lon10.375_v100  78888 non-null  float64
 7   lat54.125_lon10.375_d2m   78888 non-null  float64
 8   lat54.125_lon10.375_t2m   78888 non-null  float64
 9   lat54.125_lon10.375_ssrd  78888 non-null  float64
 10  lat54.125_lon13.375_u100  78888 non-null  float64
 11  lat54.125_lon13.375_v100  78888 non-null  float64
 12  lat54.125_lon13.375_d2m   78888 non-null  float64
 13  lat54.125_lon13.375_t2m   

In [15]:
df_final.describe().T

,count,mean,std,min,25%,50%,75%,max
lat54.125_lon7.375_u100,78888.0,2.636858,6.720222,-22.034622,-2.077981,3.161299,7.465008,2.527547e+01
lat54.125_lon7.375_v100,78888.0,1.331799,5.907846,-21.244490,-2.979716,1.202114,5.535081,2.414250e+01
lat54.125_lon7.375_d2m,78888.0,280.625215,5.299612,261.743940,276.630053,280.381758,284.954591,2.926638e+02
lat54.125_lon7.375_t2m,78888.0,283.810690,5.361117,267.078705,279.499171,283.387416,288.437295,2.994362e+02
lat54.125_lon7.375_ssrd,78888.0,465580.824634,714013.917356,-1.901566,0.000000,21448.239452,713144.209721,3.118329e+06
lat54.125_lon10.375_u100,78888.0,2.290818,5.702456,-18.133621,-1.971001,2.913924,6.556459,2.134478e+01
lat54.125_lon10.375_v100,78888.0,1.395554,4.338471,-15.570334,-1.817051,1.198923,4.550439,1.877744e+01
lat54.125_lon10.375_d2m,78888.0,279.790028,5.752728,258.736593,275.401304,279.783595,284.442555,2.933528e+02
lat54.125_lon10.375_t2m,78888.0,283.304503,6.635329,262.617086,278.127864,282.923680,288.449208,3.055852e+02
lat54.125_lon10.375_ssrd,78888.0,447630.420655,689543.931267,-1.901566,0.000000,20281.117640,688918.606916,3.085011e+06
